In [1]:
from pathlib import Path
import sys
import json

# Поднимаемся на два уровня вверх (из notebooks/ в InteriorClass/)
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))  # Теперь Python увидит src/

from src.config import RANDOM_SEED, SPLIT_RATIO, MIN_VAL_TEST_PER_CLASS, CLASS_LABELS
from src.dataset.splitter import DatasetSplitter
from src.dataset.interior_dataset import InteriorDataset, get_transforms
from src.models.interior_classifier_EfficientNet_B3 import InteriorClassifier
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

In [2]:
# 1. Собираем все пути
current_dir = Path.cwd()
root_project = current_dir.parent.parent
data_dir = root_project / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"

samples = InteriorDataset.collect_samples(dataset_dir=dataset_dir)

data_dir: /home/little-garden/CodeProjects/InteriorClass/data


In [3]:
print(len(samples))
print(samples[0])

55410
(PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/A0/A0_00000_f565.jpg'), 0)


In [4]:
# Создание сплиттера
splitter = DatasetSplitter(
    class_labels=["A0", "A1", "B0", "B1", "C0", "C1", "D0", "D1"],
    split_config=SPLIT_RATIO,
    random_seed=RANDOM_SEED
)

# Разделение данных
train_samples, val_samples, test_samples = splitter.split(samples, shuffle=True)

In [5]:
print(len(train_samples))
print(len(val_samples))
print(len(test_samples))

48985
3206
3219


In [6]:
# Конфигурация
batch_size = 32
epochs = 25
lr = 3e-5
img_size = 380  # Для EfficientNet-B3

# Datasets
train_dataset = InteriorDataset(
    train_samples,
    transform=get_transforms(mode='train'),
    mode='train'
)

val_dataset = InteriorDataset(
    val_samples,
    transform=get_transforms(mode='val'),
    mode='val'
)

test_dataset = InteriorDataset(
    test_samples,
    transform=get_transforms(mode='test'),
    mode='test'
)


# DataLoaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [7]:
def train():
    # Пути для сохранения результатов
    exp_dir = root_project / Path("experiments/exp001_baseline/results")
    exp_dir.mkdir(parents=True, exist_ok=True)
    
    # Файлы для сохранения
    checkpoint_path = exp_dir / "best_model.pth"
    log_path = exp_dir / "training_log.json"
    
    # Инициализация лога
    if log_path.exists():
        with open(log_path, "r") as f:
            log = json.load(f)
        best_val_loss = log.get("best_val_loss", float("inf"))
    else:
        log = {"train_loss": [], "val_loss": [], "val_accuracy": [], "best_val_loss": float("inf")}
        best_val_loss = float("inf")
    
    # Модель и оптимизатор
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = InteriorClassifier(num_classes=len(InteriorDataset.CLASSES)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    # Цикл обучения
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0

        # Создаем прогресс-бар с дополнительной информацией
        train_bar = tqdm(
            train_loader,
            desc=f'Epoch {epoch}/{epochs} [Train]',
            postfix={'loss': '?', 'lr': optimizer.param_groups[0]['lr']},
            bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}'
        )
        
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)

            # Обновляем постфикс в реальном времени
            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'lr': f"{optimizer.param_groups[0]['lr']:.2e}"
            })
        
        scheduler.step()
        
        # Валидация
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        
        val_bar = tqdm(
            val_loader,
            desc=f'Epoch {epoch}/{epochs} [Val]',
            bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}'
        )

        with torch.no_grad():
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                val_bar.set_postfix({'val_loss': f'{loss.item():.4f}'})
        
        # Метрики
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)

        # Сохраняем метрики в лог
        log["train_loss"].append(train_loss)
        log["val_loss"].append(val_loss)
        
        # Красивый вывод метрик
        print(f"\nEpoch {epoch} Summary:")
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Компактный отчет без лишней информации
        report = classification_report(
            all_labels, all_preds, 
            target_names=InteriorDataset.CLASSES,
            zero_division=0,
            digits=4,
            output_dict=True
        )
        
        val_accuracy = report['accuracy']
        log["val_accuracy"].append(val_accuracy)
        
        # Выводим только accuracy и макро-усредненные метрики
        print(f"Val Accuracy: {val_accuracy:.4f}")
        print(f"Macro Avg: P={report['macro avg']['precision']:.4f} "
            f"R={report['macro avg']['recall']:.4f} "
            f"F1={report['macro avg']['f1-score']:.4f}\n")
        
        # Сохраняем модель если улучшился val_loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            log["best_val_loss"] = best_val_loss
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_loss,
                'accuracy': val_accuracy,
            }, checkpoint_path)
            
            print(f"Checkpoint saved to {checkpoint_path} (Val Loss improved to {val_loss:.4f})")
        
        # Сохраняем логи
        with open(log_path, "w") as f:
            json.dump(log, f, indent=4)
    
    # Финальная оценка на тестовом наборе
    model.eval()
    test_preds, test_labels = [], []

    test_bar = tqdm(
        test_loader, 
        desc='Final Testing',
        bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}'
    )
    
    with torch.no_grad():
        for inputs, labels in test_bar:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
    
    # Компактный финальный отчет
    print("\nFinal Test Results:")
    final_report = classification_report(
        test_labels, test_preds,
        target_names=InteriorDataset.CLASSES,
        digits=4,
        output_dict=True
    )

    test_accuracy = final_report['accuracy']
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Macro Avg: P={final_report['macro avg']['precision']:.4f} "
        f"R={final_report['macro avg']['recall']:.4f} "
        f"F1={final_report['macro avg']['f1-score']:.4f}")
    
    # Сохраняем финальные результаты в лог
    log["test_accuracy"] = test_accuracy
    log["test_report"] = final_report
    with open(log_path, "w") as f:
        json.dump(log, f, indent=4)

    return model


In [ ]:
model = train()

Epoch 1/25 [Train]:  42%|████████▎           | 640/1530 [01:37<02:14,  6.60it/s, loss=0.1467, lr=3.00e-05]/home/little-garden/CodeProjects/InteriorClass/venv3.12/lib/python3.12/site-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/25 [Val]: 100%|████████████████████| 101/101 [00:04<00:00, 22.81it/s, val_loss=0.0744]



Epoch 1 Summary:
Train Loss: 0.3553 | Val Loss: 0.2847
Val Accuracy: 0.8993
Macro Avg: P=0.7086 R=0.6203 F1=0.6099

Checkpoint saved to /home/little-garden/CodeProjects/InteriorClass/experiments/exp001_baseline/results/best_model.pth (Val Loss improved to 0.2847)


Epoch 2/25 [Train]:   7%|█▎                  | 100/1530 [00:15<03:36,  6.61it/s, loss=0.1620, lr=2.99e-05]